# Analysis of a PINN trained with several parameters

This has been in the course of master thesis and further work for the publication

In [2]:
print('helloo')

helloo


In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


In [4]:
def plotthis(m,c):
    x = np.random.rand(10)
    y = m *x + c
    plt.plot(x,y)
    y = c *x + m
    plt.plot(x,y)
    plt.show()

from ipywidgets import interact, interactive, fixed, interact_manual


interactive(plotthis, m=(-10,10, 0.5), c=(-5,5,0.5))



interactive(children=(FloatSlider(value=0.0, description='m', max=10.0, min=-10.0, step=0.5), FloatSlider(valu…

In [5]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as mc
import colorsys
from matplotlib import rc
import matplotlib.animation as animation
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision import transforms


In [7]:
#Setting file paths
folder_path = "D:\polybox\Masterthesis\PINN\Taniya\Paper_Model_param_6_24/"
images_path = folder_path + "Images/"
model_path = folder_path + "TrainedModel/"
val_path =  "D:\polybox\Masterthesis\PINN\Taniya\\fem\\"
#loading trained model to evaluate
model = torch.load("model_best_direct.pkl", torch.device('cpu'))
model=model.eval()

In [8]:
#setting model constants
umax=3000
tmax=0.0015
xmax=1.8
ymax=2
cond=0.1
rhocp=0.0049731
pow=0.075
v=1
sig=0.05
extrema =torch.tensor([[-0.1, 1],[-1, 1.8], [-1, 1], [-1,0.03]])

#Setting plotting values
plot_dim=100 #resolution
t1 = 0.0 #plot at time t=t1
plotsize = 0.5 #size of markers
z1 = 0.03 #evaluated z-height for x-y-plots

In [9]:

#set dimension of plots
'''height=50
width=50
plt.rcParams['figure.figsize'] = [height, width]
plt.rcParams.update({'font.family':'DejaVu Sans'})
plot_dpi=1800'''


"height=50\nwidth=50\nplt.rcParams['figure.figsize'] = [height, width]\nplt.rcParams.update({'font.family':'DejaVu Sans'})\nplot_dpi=1800"

In [10]:

# read in data set of FEM
node_coord_df = pd.read_csv(val_path + 'nodes.csv', index_col='label')
temp_df = pd.read_csv(val_path + 'temp100.csv', index_col='label')
test_inp = node_coord_df.to_numpy()
Exact = temp_df.to_numpy()

In [27]:
def calc_meltpool_dim(timestep, material, P, v):
    P=P/1000
    if material==1:
        cond=0.0072
        rhocp=0.0025
        T_solidus=1640
    elif material==2:
        cond=0.015
        rhocp=0.004
        T_solidus=1370
    elif material==3:
        cond=0.001
        rhocp=0.004
        T_solidus=1300
    t_i = float(0.0001 * timestep / 0.0015)
    
    x_source = t_i * 1.5  # x-coordinate of source at evaluated timestep for further use
    # Evaluate PINN Model Prediction in x-y
    imax = 10000
    ### Meltpool dimesnion 1: x+
    i = 0
    x = x_source
    T_pred = 3000
    inc = 0.05
    
    while i < imax:
        i = i + 1
        x = np.array(x + inc, dtype=np.float32)
        y = np.array(0, dtype=np.float32)
        z = np.array(z1, dtype=np.float32)
        x_t = torch.reshape(torch.from_numpy(x), [np.size(x), 1])
        y_t = torch.reshape(torch.from_numpy(y), [np.size(x), 1])
        z_t = torch.reshape(torch.from_numpy(z), [np.size(x), 1])
        t_t = torch.tensor(()).new_full(size=(np.size(x), 1), fill_value=t_i)
        p_c = torch.cat(
        [torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=cond),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=rhocp),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=P),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=v)]
        , 1)
        plot_var2 = torch.cat([t_t, x_t, y_t, z_t, p_c], 1)
        T_pred = model(plot_var2)[:, 0]
        T_pred = T_pred.detach().cpu().numpy() * umax
        if T_pred < T_solidus:
            x = x - inc
            inc = inc / 2
        if inc < 0.000001:
            i = imax
    print("Melpool size in front of Laser", round(x - x_source, 5))
    x_max = x
    ### Meltpool dimesnion 2: x-
    i = 0
    x = x_source
    t_t = torch.tensor(()).new_full(size=(np.size(x), 1), fill_value=t_i)
    T_pred = 3000
    inc = 0.1
    while i < imax:
        i = i + 1
        x = np.array(x - inc, dtype=np.float32)
        y = np.array(0, dtype=np.float32)
        z = np.array(z1, dtype=np.float32)
        x_t = torch.reshape(torch.from_numpy(x), [np.size(x), 1])
        y_t = torch.reshape(torch.from_numpy(y), [np.size(x), 1])
        z_t = torch.reshape(torch.from_numpy(z), [np.size(x), 1])
        plot_var2 = torch.cat([t_t, x_t, y_t, z_t, p_c], 1)
        T_pred = model(plot_var2)[:, 0]
        T_pred = T_pred.detach().cpu().numpy() * umax
        if T_pred < T_solidus:
            x = x + inc
            inc = inc / 2
        if inc < 0.000001:
            i = imax
    x_min = x

    ### Meltpool dimesnion 3: y+
    print("Melpool size behind Laser", round(abs(x_min - x_source), 5))
    i = 0
    y = 0
    T_pred = 3000
    inc = 0.1
    while i < imax:
        i = i + 1
        x = np.array(x_source, dtype=np.float32)
        y = np.array(y + inc, dtype=np.float32)
        z = np.array(z1, dtype=np.float32)
        x_t = torch.reshape(torch.from_numpy(x), [np.size(x), 1])
        y_t = torch.reshape(torch.from_numpy(y), [np.size(x), 1])
        z_t = torch.reshape(torch.from_numpy(z), [np.size(x), 1])
        plot_var2 = torch.cat([t_t, x_t, y_t, z_t,p_c], 1)
        T_pred = model(plot_var2)[:, 0]
        T_pred = T_pred.detach().cpu().numpy() * umax
        if T_pred < T_solidus:
            y = y - inc
            inc = inc / 2
        if inc < 0.000001:
            i = imax
    y_max = y
    print("Melpool size left of Laser", round(y, 5))
    i = 0
    y = 0
    T_pred = 3000
    inc = 0.1
    while i < imax:
        i = i + 1
        x = np.array(x_source, dtype=np.float32)
        y = np.array(y - inc, dtype=np.float32)
        z = np.array(z1, dtype=np.float32)
        x_t = torch.reshape(torch.from_numpy(x), [np.size(x), 1])
        y_t = torch.reshape(torch.from_numpy(y), [np.size(x), 1])
        z_t = torch.reshape(torch.from_numpy(z), [np.size(x), 1])
        plot_var2 = torch.cat([t_t, x_t, y_t, z_t, p_c], 1)
        T_pred = model(plot_var2)[:, 0]
        T_pred = T_pred.detach().cpu().numpy() * umax
        if T_pred < T_solidus:
            y = y + inc
            inc = inc / 2
        if inc < 0.000001:
            i = imax
    y_min = y
    print("Melpool size right of Laser", round(abs(y), 5))
    i = 0
    T_pred = 3000
    inc = 0.1
    while i < imax:
        i = i + 1
        x = np.array(x_source, dtype=np.float32)
        y = np.array(0, dtype=np.float32)
        z = np.array(z - inc, dtype=np.float32)
        x_t = torch.reshape(torch.from_numpy(x), [np.size(x), 1])
        y_t = torch.reshape(torch.from_numpy(y), [np.size(x), 1])
        z_t = torch.reshape(torch.from_numpy(z), [np.size(x), 1])
        plot_var2 = torch.cat([t_t, x_t, y_t, z_t, p_c], 1)
        T_pred = model(plot_var2)[:, 0]
        T_pred = T_pred.detach().cpu().numpy() * umax
        if T_pred < T_solidus:
            z = z + inc
            inc = inc / 2
        if inc < 0.000001:
            i = imax
    z_min = z
    print("Melpool size below Laser", round(z1-z_min, 5))

In [36]:
def plot_model(timestep, material, P, v, zoom):
    
    if material==1:
        cond=0.0072
        rhocp=0.0025
    elif material==2:
        cond=0.015
        rhocp=0.004
    elif material==3:
        cond=0.001
        rhocp=0.004
    
    P=P/1000
    t_i = float(0.0001 * timestep / 0.0015)
    sig = 0.05
    y = torch.reshape(torch.linspace(extrema[2, 0]/zoom, extrema[2, 1]/zoom, plot_dim), [plot_dim, 1])
    x = torch.reshape(torch.linspace( 1.5-2.8*torch.max(y).numpy().round(2), 1.5, plot_dim), [plot_dim, 1])
    # Plot the surface.
    X = x.numpy()
    Y = y.numpy()
    X, Y = np.meshgrid(X, Y)
    x_t = torch.reshape(torch.from_numpy(X), [plot_dim ** 2, 1])
    y_t = torch.reshape(torch.from_numpy(Y), [plot_dim ** 2, 1])
    p_c = torch.cat(
        [torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=cond),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=rhocp),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=P),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=v)]
        , 1)

    plot_var = torch.cat(
                        [torch.tensor(()).new_full(size=(plot_dim ** 2, 1), fill_value=t_i),
                        x_t,
                        y_t,
                        torch.tensor(()).new_full(size=(plot_dim ** 2, 1), fill_value=z1),
                        p_c],1)
    Z4 = model(plot_var)[:, 0]
    Z4 = torch.reshape(Z4, [plot_dim, plot_dim])
    Z4 = Z4.detach().cpu().numpy() * umax    
    print(np.max(Z4))
    fig, (ax2) = plt.subplots(ncols=1) 
    sns.heatmap(Z4, cmap="rocket", ax=ax2, cbar=False)
    
    #specify x-axis tick positions and labels
    plt.xticks([0, plot_dim/2, plot_dim], [torch.min(x).numpy().round(2),np.round((torch.max(x).numpy()-torch.min(x).numpy().round(2))/2,2), torch.max(x).numpy().round(2)])
    #specify y-axis tick positions and labels
    plt.yticks([0, plot_dim/2, plot_dim], [torch.min(y).numpy().round(2),0 , torch.max(y).numpy().round(2)])
    fig.colorbar(ax2.collections[0], ax=ax2, location="right", use_gridspec=False, pad=0.2)

    ax2.set_title('Predicted Temperature')
    plt.show()


In [37]:
from ipywidgets import interact_manual#t_i, cond, rhocp, P, v, T_solidus
T_solidus=1260
interact_manual(plot_model,timestep=(0,15, 1), material=[('Titanium', 1), ('Stainless', 2), ('HastelloyX',3)], P=(50,200,10), v=(0.2,2,0.02), zoom=(1,5,1))


interactive(children=(IntSlider(value=7, description='timestep', max=15), Dropdown(description='material', opt…

<function __main__.plot_model(timestep, material, P, v, zoom)>

In [30]:

interact_manual(calc_meltpool_dim,timestep=(0,15, 1), material=[('Titanium', 1), ('Stainless', 2), ('HastelloyX',3)], P=(50,200,10), v=(0.2,2,0.02), zoom=(1,5,1))


interactive(children=(IntSlider(value=7, description='timestep', max=15), Dropdown(description='material', opt…

<function __main__.calc_meltpool_dim(timestep, material, P, v)>

In [33]:
def plot_melt(timestep, material, P, v, zoom):
    
    if material==1:
        cond=0.0072
        rhocp=0.0025
        T_solidus=1640
    elif material==2:
        cond=0.015
        rhocp=0.004
        T_solidus=1370
    elif material==3:
        cond=0.001
        rhocp=0.004
        T_solidus=1300
    
    P=P/1000
    t_i = float(0.0001 * timestep / 0.0015)
    sig = 0.05
    y = torch.reshape(torch.linspace(extrema[2, 0]/zoom, extrema[2, 1]/zoom, plot_dim), [plot_dim, 1])
    x = torch.reshape(torch.linspace( 1.5-2.8*torch.max(y).numpy().round(2), 1.5, plot_dim), [plot_dim, 1])
    # Plot the surface.
    X = x.numpy()
    Y = y.numpy()
    X, Y = np.meshgrid(X, Y)
    x_t = torch.reshape(torch.from_numpy(X), [plot_dim ** 2, 1])
    y_t = torch.reshape(torch.from_numpy(Y), [plot_dim ** 2, 1])
    p_c = torch.cat(
        [torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=cond),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=rhocp),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=P),
         torch.tensor(()).new_full(size=(x_t.shape[0], 1), fill_value=v)]
        , 1)

    plot_var = torch.cat(
                        [torch.tensor(()).new_full(size=(plot_dim ** 2, 1), fill_value=t_i),
                        x_t,
                        y_t,
                        torch.tensor(()).new_full(size=(plot_dim ** 2, 1), fill_value=z1),
                        p_c],1)
    Z4 = model(plot_var)[:, 0]
    Z4 = torch.reshape(Z4, [plot_dim, plot_dim])
    Z4 = Z4.detach().cpu().numpy() * umax    
    Melt = Z4 > T_solidus
    fig, (ax2) = plt.subplots(ncols=1) 
    sns.heatmap(Melt, cmap="rocket", ax=ax2, cbar=False)
    
    #specify x-axis tick positions and labels
    plt.xticks([0, plot_dim/2, plot_dim], [torch.min(x).numpy().round(2),np.round((torch.max(x).numpy()-torch.min(x).numpy().round(2))/2,2), torch.max(x).numpy().round(2)])
    #specify y-axis tick positions and labels
    plt.yticks([0, plot_dim/2, plot_dim], [torch.min(y).numpy().round(2),0 , torch.max(y).numpy().round(2)])
    fig.colorbar(ax2.collections[0], ax=ax2, location="right", use_gridspec=False, pad=0.2)

    ax2.set_title('Predicted Temperature')
    plt.show()

In [34]:

interact_manual(plot_melt,timestep=(0,15, 1), material=[('Titanium', 1), ('Stainless', 2), ('HastelloyX',3)], P=(50,200,10), v=(0.2,2,0.02), zoom=(1,5,1))


interactive(children=(IntSlider(value=7, description='timestep', max=15), Dropdown(description='material', opt…

<function __main__.plot_melt(timestep, material, P, v, zoom)>